In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
from glob import glob

import numpy as np
import pandas as pd

import settings as conf

# Load gene mappings

In [3]:
with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-0.pkl'), 'rb') as f:
    genes_mapping_0 = pickle.load(f)

with open(os.path.join(conf.GENES_METADATA_DIR, 'genes_mapping_simplified-1.pkl'), 'rb') as f:
    genes_mapping_1 = pickle.load(f)

# Load S-MultiXcan results

In [4]:
# s-predixcan
smultixcan_genes_associations_filename = os.path.join(conf.GENE_ASSOC_DIR, f'smultixcan-mashr-pvalues.pkl')
display(smultixcan_genes_associations_filename)

smultixcan_genes_associations = pd.read_pickle(smultixcan_genes_associations_filename)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/gene_assoc/smultixcan-mashr-pvalues.pkl'

In [5]:
smultixcan_genes_associations.shape

(22515, 4091)

In [6]:
smultixcan_genes_associations.head(5)

,20096_1-Size_of_red_wine_glass_drunk_small_125ml,2345-Ever_had_bowel_cancer_screening,N49-Diagnoses_main_ICD10_N49_Inflammatory_disorders_of_male_genital_organs_not_elsewhere_classified,100011_raw-Iron,5221-Index_of_best_refractometry_result_right,20003_1141150624-Treatmentmedication_code_zomig_25mg_tablet,S69-Diagnoses_main_ICD10_S69_Other_and_unspecified_injuries_of_wrist_and_hand,20024_1136-Job_code_deduced_Information_and_communication_technology_managers,20002_1385-Noncancer_illness_code_selfreported_allergy_or_anaphylactic_reaction_to_food,G6_SLEEPAPNO-Sleep_apnoea,...,Astle_et_al_2016_Sum_basophil_neutrophil_counts,RA_OKADA_TRANS_ETHNIC,pgc.scz2,PGC_ADHD_EUR_2017,MAGIC_FastingGlucose,Astle_et_al_2016_Red_blood_cell_count,SSGAC_Depressive_Symptoms,BCAC_ER_positive_BreastCancer_EUR,IBD.EUR.Inflammatory_Bowel_Disease,Astle_et_al_2016_High_light_scatter_reticulocyte_count
gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000000419,0.865429,0.918314,0.810683,0.374671,0.189032,0.140981,0.467741,0.129427,0.193680,0.285479,...,0.416210,0.782554,0.609467,0.980281,0.666504,0.409761,0.713310,0.168319,0.460244,0.765506
ENSG00000000457,0.174192,0.064765,0.889194,0.896938,0.448596,0.269602,0.540261,0.068405,0.041813,0.313427,...,0.149360,0.512603,0.010907,0.228982,0.607081,0.812484,0.678749,0.918971,0.311187,0.344574
ENSG00000000460,0.879969,0.240715,0.238228,0.567555,0.921320,0.825036,0.782230,0.644525,0.392273,0.840014,...,0.503520,0.764147,0.587969,0.301460,0.629621,0.486664,0.736509,0.933600,0.000477,0.321223
ENSG00000000938,0.192670,0.400054,0.114353,0.470700,0.889202,0.000011,0.899764,0.212352,0.829671,0.372348,...,0.899212,0.961678,0.059247,0.588855,0.898525,0.135045,0.954998,0.088220,0.176497,0.304281
ENSG00000000971,0.180632,0.793060,0.490585,0.088752,0.744531,0.949639,0.253817,0.377408,0.971655,0.070266,...,0.390618,0.093824,0.020391,0.109883,0.870551,0.995450,0.002660,0.421588,0.656851,0.868416


# Create UKB data with traits

In [7]:
ukb_data = pd.DataFrame(index=smultixcan_genes_associations.columns).reset_index().rename(columns={'index': 'ukb_fullcode'})

In [8]:
ukb_data = ukb_data.assign(ukb_code=ukb_data['ukb_fullcode'].str.split('-').str[0])

In [9]:
ukb_data.head()

,ukb_fullcode,ukb_code
0,20096_1-Size_of_red_wine_glass_drunk_small_125ml,20096_1
1,2345-Ever_had_bowel_cancer_screening,2345
2,N49-Diagnoses_main_ICD10_N49_Inflammatory_diso...,N49
3,100011_raw-Iron,100011_raw
4,5221-Index_of_best_refractometry_result_right,5221


# Take a look at the UKB-EFO mappings file

**Note:** `UK_Biobank_master_file.tsv` was downloaded from https://github.com/EBISPOT/EFO-UKB-mappings

In [10]:
ukb_map = pd.read_csv(conf.OMIM_SILVER_STANDARD_UKB_EFO_MAP_FILE, sep='\t')

In [11]:
ukb_map = ukb_map.dropna(subset=['ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE'])

In [12]:
ukb_map.shape

(1564, 7)

In [13]:
ukb_map.head()

,ZOOMA QUERY,MAPPED_TERM_LABEL,MAPPED_TERM_URI,MAPPING_TYPE,ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE,COMMENTS/TICKET,AI
0,Vascular disorders of intestine,vascular disease,"EFO_0004264, EFO_0009431",Broad,K55,DONE,NaN
1,Gonarthrosis,osteoarthritis || knee,EFO_0004616,Broad,M17,DONE,NaN
2,Psoriatic and enteropathic arthropathies,psoriatic arthritis,EFO_0003778,? Broad,M07,DONE,NaN
3,Pain associated with micturition,dysuria,EFO_0003901,? Broad,R30,DONE,NaN
4,Other mood,mood disorder,EFO_0004247,? Broad,F38,DONE,NaN


In [14]:
ukb_map['MAPPING_TYPE'].unique()

array(['Broad', '? Broad', '? Exact', '? Narrow', 'Exact', 'Narrow', '?',
       nan, 'Narrow?'], dtype=object)

In [15]:
ukb_map[ukb_map['MAPPING_TYPE'] == 'Narrow'].shape

(50, 7)

In [16]:
ukb_map[ukb_map['MAPPING_TYPE'] == 'Broad'].shape

(572, 7)

In [17]:
ukb_map[ukb_map['MAPPING_TYPE'] == 'Exact'].shape

(927, 7)

In [18]:
ukb_map[ukb_map['ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE'].str.contains('20001_1073')]

,ZOOMA QUERY,MAPPED_TERM_LABEL,MAPPED_TERM_URI,MAPPING_TYPE,ICD10_CODE/SELF_REPORTED_TRAIT_FIELD_CODE,COMMENTS/TICKET,AI
992,rodent ulcer,basal cell carcinoma,EFO_0004193,?,20001_1073,DONE,NaN


# Group traits according to EFO terms

In [19]:
from efo import EFO

In [20]:
efo_file = os.path.join(conf.OMIM_SILVER_STANDARD_EFO_FILE)
display(efo_file)

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/omim_silver_standard/data/EFO.csv.gz'

In [21]:
efo = EFO(efo_file)

In [31]:
efo.efo_data_full.head(10)

,class_id,omim_codes,preferred_label
0,http://www.orpha.net/ORDO/Orphanet_1390,UMLS:CN199356|MONDO:0015326|GARD:0003994|ICD10...,Night blindness - skeletal anomalies - dysmorp...
1,http://www.ebi.ac.uk/efo/EFO_1000880,SNOMEDCT:19161004|MSH:D016510|MONDO:0006713|IC...,corneal neovascularization
2,http://www.ebi.ac.uk/efo/EFO_0004381,NCIt:C48555,mole per liter
3,http://www.orpha.net/ORDO/Orphanet_69735,OMIM:607823|GARD:0012827|MONDO:0007670,Hypotrichosis - lymphedema - telangiectasia
4,http://purl.obolibrary.org/obo/GO_0120062,NaN,positive regulation of gastric emptying
5,http://www.ebi.ac.uk/efo/EFO_1000828,MSH:D015456|DOID:9953|MONDO:0006667|ICD10:C95.0,B- and T-cell mixed leukemia
6,http://purl.obolibrary.org/obo/NCBITaxon_5076,SNOMEDCT:24406000|MSH:D010408,Penicillium chrysogenum
7,http://www.ebi.ac.uk/efo/EFO_0008234,NaN,MHC class I polypeptide-related sequence B mea...
8,http://purl.obolibrary.org/obo/CHEBI_53115,MSH:C081320,8-(3-chlorostyryl) caffeine
9,http://purl.obolibrary.org/obo/CHEBI_17768,NaN,N-acetylputrescine


In [38]:
# ukb_data_with_efo = efo.assign_efo_from_ukb(ukb_data, map_types=('Exact',))
ukb_data_with_efo = efo.assign_efo_from_ukb(ukb_data, map_types=('Exact', 'Broad', 'Narrow'))

In [39]:
ukb_data_with_efo = ukb_data_with_efo.dropna(subset=['efo_name'])

In [40]:
display(ukb_data_with_efo.shape)

(1007, 4)

In [41]:
display(ukb_data_with_efo.head())

,ukb_fullcode,ukb_code,efo_code,efo_name
2,N49-Diagnoses_main_ICD10_N49_Inflammatory_diso...,N49,EFO_0009555,male reproductive system disease
6,S69-Diagnoses_main_ICD10_S69_Other_and_unspeci...,S69,EFO_0000546,injury
8,20002_1385-Noncancer_illness_code_selfreported...,20002_1385,EFO_1001890,food allergy
13,20002_1373-Noncancer_illness_code_selfreported...,20002_1373,EFO_1001986,connective tissue disease
15,20002_1510-Noncancer_illness_code_selfreported...,20002_1510,EFO_0008533,dyspepsia


In [42]:
# How does "asthma" get grouped?
ukb_data_with_efo[ukb_data_with_efo['efo_name'].str.lower().str.contains('asthma')]

,ukb_fullcode,ukb_code,efo_code,efo_name
991,J46-Diagnoses_main_ICD10_J46_Status_asthmaticus,J46,EFO_0008590,Status Asthmaticus
1213,20002_1111-Noncancer_illness_code_selfreported...,20002_1111,EFO_0000270,asthma
3485,22127-Doctor_diagnosed_asthma,22127,EFO_0000270,asthma
3667,J45-Diagnoses_main_ICD10_J45_Asthma,J45,EFO_0000270,asthma


In [43]:
# Save only ukb traits
ukb_data_with_efo.to_csv('ukbiobank_efo_mappings.tsv', index=False, sep='\t')

# Add GTEX GWAS traits description

In [29]:
import metadata

In [30]:
metadata.GTEX_GWAS_PHENO_INFO.head()

,GTEx_GWAS,PUBMED_Paper_Link,Pheno_File,Source_File,Portal,Consortium,Link,Notes,Header,Hidden,...,Declared_Effect_Allele,Genome_Reference,Binary,Cases,abbreviation,new_abbreviation,new_Phenotype,Category,Deflation,color
Tag,,,,,,,,,,,,,,,,,,,,,
UKB_1160_Sleep_duration,Yes,http://biobank.ctsu.ox.ac.uk/showcase/field.cg...,1160.assoc.tsv.gz,1160.assoc.tsv.gz,http://biobank.ctsu.ox.ac.uk/,UK Biobank,https://www.dropbox.com/s/00v10oxwku3u5yq/1160...,NaN,NaN,NaN,...,NaN,NaN,0,NaN,U_SL,SLEEP_UKB,Sleep_Duration_UKB,Psychiatric-neurologic,0,#0000A0
UKB_1180_Morning_or_evening_person_chronotype,Yes,http://biobank.ctsu.ox.ac.uk/showcase/field.cg...,1180.assoc.tsv.gz,1180.assoc.tsv.gz,http://biobank.ctsu.ox.ac.uk/,UK Biobank,https://www.dropbox.com/s/z7cppp7ndmylpf0/1180...,NaN,NaN,NaN,...,NaN,NaN,0,NaN,U_CHR,CHRONO_UKB,Chronotype_UKB,Psychiatric-neurologic,0,#0000A0
UKB_1200_Sleeplessness_or_insomnia,Yes,http://biobank.ctsu.ox.ac.uk/showcase/field.cg...,1200.assoc.tsv.gz,1200.assoc.tsv.gz,http://biobank.ctsu.ox.ac.uk/,UK Biobank,https://www.dropbox.com/s/ugt1g44ycju9ms5/1200...,NaN,NaN,NaN,...,NaN,NaN,0,NaN,U_NSL,INSOMN_UKB,Insomnia_UKB,Psychiatric-neurologic,0,#0000A0
UKB_1807_Fathers_age_at_death,Yes,http://biobank.ctsu.ox.ac.uk/showcase/field.cg...,1807.assoc.tsv.gz,1807.assoc.tsv.gz,http://biobank.ctsu.ox.ac.uk/,UK Biobank,https://www.dropbox.com/s/qftvyi5a9qkb5xw/1807...,NaN,NaN,NaN,...,NaN,NaN,0,NaN,U_FAD,FAD_UKB,Fathers_Age_At_Death_UKB,Aging,0,#400040
UKB_20002_1094_self_reported_deep_venous_thrombosis_dvt,Yes,http://biobank.ctsu.ox.ac.uk/showcase/field.cg...,20002_1094.assoc.tsv.gz,20002_1094.assoc.tsv.gz,http://biobank.ctsu.ox.ac.uk/,UK Biobank,https://www.dropbox.com/s/0smh60npg2eihqv/2000...,NaN,NaN,NaN,...,NaN,NaN,1,6767.0,UN_DVT,DVT_UKBS,Deep_Venous_Thrombosis_UKBS,Cardiometabolic,0,#004000


In [31]:
gtex_gwas_traits_info = metadata.GTEX_GWAS_PHENO_INFO[['EFO', 'HPO', 'Description', 'Phenotype']]

In [32]:
gtex_gwas_traits_info = gtex_gwas_traits_info[gtex_gwas_traits_info.index.isin(smultixcan_genes_associations.columns)]

In [33]:
gtex_gwas_traits_info.shape

(42, 4)

In [34]:
gtex_gwas_traits_info.head()

,EFO,HPO,Description,Phenotype
Tag,,,,
Astle_et_al_2016_Eosinophil_counts,EFO_0004842,NaN,NaN,Eosinophil counts
Astle_et_al_2016_Granulocyte_count,EFO_0007987,NaN,NaN,Granulocyte count
Astle_et_al_2016_High_light_scatter_reticulocyte_count,EFO_0007986,NaN,NaN,High light scatter reticulocyte count
Astle_et_al_2016_Lymphocyte_counts,EFO_0004587,NaN,NaN,Lymphocyte counts
Astle_et_al_2016_Monocyte_count,EFO_0005091,NaN,NaN,Monocyte count


In [35]:
gtex_gwas_data_with_efo = efo.assign_efo_label_from_efo_code(gtex_gwas_traits_info, efo_code_column='EFO')

In [36]:
gtex_gwas_data_with_efo.head()

,EFO,HPO,Description,Phenotype,efo_name
Tag,,,,,
Astle_et_al_2016_Eosinophil_counts,EFO_0004842,NaN,NaN,Eosinophil counts,eosinophil count
Astle_et_al_2016_Granulocyte_count,EFO_0007987,NaN,NaN,Granulocyte count,granulocyte count
Astle_et_al_2016_High_light_scatter_reticulocyte_count,EFO_0007986,NaN,NaN,High light scatter reticulocyte count,reticulocyte count
Astle_et_al_2016_Lymphocyte_counts,EFO_0004587,NaN,NaN,Lymphocyte counts,lymphocyte count
Astle_et_al_2016_Monocyte_count,EFO_0005091,NaN,NaN,Monocyte count,monocyte count


In [37]:
gtex_gwas_data_with_efo = gtex_gwas_data_with_efo[['efo_name']]

In [38]:
gtex_gwas_data_with_efo = gtex_gwas_data_with_efo.assign(ukb_code=gtex_gwas_data_with_efo.index)

In [39]:
gtex_gwas_data_with_efo = gtex_gwas_data_with_efo.reset_index()[['Tag', 'ukb_code', 'efo_name']].rename(columns={'Tag': 'ukb_fullcode'})

In [40]:
gtex_gwas_data_with_efo.head()

,ukb_fullcode,ukb_code,efo_name
0,Astle_et_al_2016_Eosinophil_counts,Astle_et_al_2016_Eosinophil_counts,eosinophil count
1,Astle_et_al_2016_Granulocyte_count,Astle_et_al_2016_Granulocyte_count,granulocyte count
2,Astle_et_al_2016_High_light_scatter_reticulocy...,Astle_et_al_2016_High_light_scatter_reticulocy...,reticulocyte count
3,Astle_et_al_2016_Lymphocyte_counts,Astle_et_al_2016_Lymphocyte_counts,lymphocyte count
4,Astle_et_al_2016_Monocyte_count,Astle_et_al_2016_Monocyte_count,monocyte count


In [41]:
df = pd.concat((ukb_data_with_efo, gtex_gwas_data_with_efo), ignore_index=True)

In [42]:
# remove efo_name with nan
df = df.dropna(subset=['efo_name'])

In [43]:
df.shape

(643, 3)

In [44]:
df.head()

,ukb_fullcode,ukb_code,efo_name
0,20002_1385-Noncancer_illness_code_selfreported...,20002_1385,food allergy
1,20002_1373-Noncancer_illness_code_selfreported...,20002_1373,connective tissue disease
2,20002_1510-Noncancer_illness_code_selfreported...,20002_1510,dyspepsia
3,O48-Diagnoses_main_ICD10_O48_Prolonged_pregnancy,O48,post term pregnancy
4,N17-Diagnoses_main_ICD10_N17_Acute_renal_failure,N17,Acute kidney injury


In [45]:
# df.to_csv('ukbiobank_efo_mappings.tsv', index=False, sep='\t')

# Run map_trait_to_hpo_and_mim.R

In [46]:
omim_silver_standard_dir = conf.OMIM_SILVER_STANDARD_DATA_DIR

In [47]:
%%bash -s "$omim_silver_standard_dir"
Rscript map_trait_to_hpo_and_mim.R $1


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



# Run mim-to-gene.R (generates OMIM silver standard)

In [48]:
%%bash -s "$omim_silver_standard_dir"
Rscript mim-to-gene.R $1


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Taking input= as a system command ('cat /home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/omim_silver_standard/data/mim2gene.txt | tail -n +5') and a variable has been used in the expression passed to `input=`. Please use fread(cmd=...). There is a security concern if you are creating an app, and the app could have a malicious user, and the app is not running in a secure envionment; e.g. the app is running as root. Please read item 5 in the NEWS file for v1.11.6 for more information and for the option to suppress this message.
Taking input= as a system command ('cat /home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/omim_silver_standard/data/genemap2.txt | head -n 16943 | tail -n

# Move OMIM silver standard file

In [49]:
new_omim_file_location = conf.OMIM_SILVER_STANDARD_FINAL_FILE

In [50]:
new_omim_file_location

'/home/miltondp/projects/labs/hakyimlab/phenomexcan/base/results/omim_silver_standard/data/omim_silver_standard.tsv'

In [51]:
%%bash -s "$new_omim_file_location"
mv omim_silver_standard.tsv $1